In [3]:
from pynq import Overlay
from pynq import allocate

import numpy as np
import time
import matplotlib.pyplot as plt

# Load the kernels
overlay = Overlay('/home/xilinx/jupyter_notebooks/preesm/top_rfi_C.bit')
mem_read = overlay.mem_read_top_rfi_C_0
mem_write = overlay.mem_write_top_rfi_C_0


# Define constants
RATE_OF_RAW_DATA_REAL_I = 409600
RATE_OF_RAW_DATA_IM_I = 409600
RATE_OF_RAW_DATA_IM_O = 409600
RATE_OF_RAW_DATA_REAL_O = 409600
RATE_OF_MAD_R_O = 409600
RATE_OF_RAW_DATA_REAL_1_O = 409600
RATE_OF_STD_R_O = 409600
RATE_OF_RAW_DATA_IM_1_O = 409600
RATE_OF_MAD_I_O = 409600
RATE_OF_STD_I_O = 409600
RATE_OF_FILTERED_IM_0_O = 409600
RATE_OF_FILTERED_REAL_0_O = 409600
RATE_OF_FILTERED_IM_1_O = 409600
RATE_OF_FILTERED_REAL_1_O = 409600

    
    
# Buffer initialization
# TODO update dtype
# see: https://numpy.org/doc/stable/user/basics.types.html
# see: https://numpy.org/doc/stable/reference/arrays.dtypes.html
# TODO fill data
filename = 'J1939_plus_2134_1152MHz.dada'
headerSize = 4096
printHeader = True
hdr = {}
fh = open(filename, 'rb')
headerBuf = fh.read(headerSize)
headerBuf = headerBuf.decode('utf-8')
for line in headerBuf.split('\n'):
    try:
        k, v = line.split(None, 1) # splits each line in the header
        hdr[k] = v # first part of each line in header is the key, other is the value for the dict
    except ValueError:
        pass

if(printHeader == True):
    print('HEADER INFO:')    
    for key, val in hdr.items():
        print(key, ':', val)
        
fs = 128e6 # sample rate in Hz
fc = int(hdr['FREQ'])*1e6 # centre frequency
nSamples = 2048
nBlocks = 200
timeSeries = np.zeros(nSamples * nBlocks,dtype=complex)
dataBufSize = 4*nSamples # size in bytes (samples are complex-values, 16-bits)
for i in range(nBlocks):
    dataBuf = fh.read(dataBufSize)
    data = np.frombuffer(dataBuf, dtype='uint16')
    # the next two lines convert the offset-binary format to Numpy floats
    data = data.astype(np.int32)
    data = (data - (2**15))
    dataCmplx = data[0::2] + 1j*data[1::2]
    timeSeries[i*nSamples:(i+1)*nSamples] = dataCmplx
fh.close()

raw_data_real_i_buff = allocate(shape=(RATE_OF_RAW_DATA_REAL_I,), dtype=np.dtype('ap_int<16>'))
raw_data_real_i_vect = np.real(timeSeries)
np.copyto(raw_data_real_i_buff, raw_data_real_i_vect)

raw_data_im_i_buff = allocate(shape=(RATE_OF_RAW_DATA_IM_I,), dtype=np.dtype('ap_int<16>'))
raw_data_im_i_vect = np.imag(timeSeries)
np.copyto(raw_data_im_i_buff, raw_data_im_i_vect)

raw_data_im_o_buff = allocate(shape=(RATE_OF_RAW_DATA_IM_O,), dtype=np.dtype('ap_int<16>'))
raw_data_real_o_buff = allocate(shape=(RATE_OF_RAW_DATA_REAL_O,), dtype=np.dtype('ap_int<16>'))
mad_R_o_buff = allocate(shape=(RATE_OF_MAD_R_O,), dtype=np.dtype('ap_int<16>'))
raw_data_real_1_o_buff = allocate(shape=(RATE_OF_RAW_DATA_REAL_1_O,), dtype=np.dtype('ap_int<16>'))
std_R_o_buff = allocate(shape=(RATE_OF_STD_R_O,), dtype=np.dtype('ap_int<16>'))
raw_data_im_1_o_buff = allocate(shape=(RATE_OF_RAW_DATA_IM_1_O,), dtype=np.dtype('ap_int<16>'))
mad_I_o_buff = allocate(shape=(RATE_OF_MAD_I_O,), dtype=np.dtype('ap_int<16>'))
std_I_o_buff = allocate(shape=(RATE_OF_STD_I_O,), dtype=np.dtype('ap_int<16>'))
filtered_im_0_o_buff = allocate(shape=(RATE_OF_FILTERED_IM_0_O,), dtype=np.dtype('ap_int<16>'))
filtered_real_0_o_buff = allocate(shape=(RATE_OF_FILTERED_REAL_0_O,), dtype=np.dtype('ap_int<16>'))
filtered_im_1_o_buff = allocate(shape=(RATE_OF_FILTERED_IM_1_O,), dtype=np.dtype('ap_int<16>'))
filtered_real_1_o_buff = allocate(shape=(RATE_OF_FILTERED_REAL_1_O,), dtype=np.dtype('ap_int<16>'))


        
# Buffer mapping to FPGA
mem_read.write(mem_read.register_map.raw_data_real_i_mem_1.address, raw_data_real_i_buff.physical_address)
mem_read.write(mem_read.register_map.raw_data_im_i_mem_1.address, raw_data_im_i_buff.physical_address)
mem_write.write(mem_write.register_map.raw_data_im_o_mem_1.address, raw_data_im_o_buff.physical_address)
mem_write.write(mem_write.register_map.raw_data_real_o_mem_1.address, raw_data_real_o_buff.physical_address)
mem_write.write(mem_write.register_map.mad_R_o_mem_1.address, mad_R_o_buff.physical_address)
mem_write.write(mem_write.register_map.raw_data_real_1_o_mem_1.address, raw_data_real_1_o_buff.physical_address)
mem_write.write(mem_write.register_map.std_R_o_mem_1.address, std_R_o_buff.physical_address)
mem_write.write(mem_write.register_map.raw_data_im_1_o_mem_1.address, raw_data_im_1_o_buff.physical_address)
mem_write.write(mem_write.register_map.mad_I_o_mem_1.address, mad_I_o_buff.physical_address)
mem_write.write(mem_write.register_map.std_I_o_mem_1.address, std_I_o_buff.physical_address)
mem_write.write(mem_write.register_map.filtered_im_0_o_mem_1.address, filtered_im_0_o_buff.physical_address)
mem_write.write(mem_write.register_map.filtered_real_0_o_mem_1.address, filtered_real_0_o_buff.physical_address)
mem_write.write(mem_write.register_map.filtered_im_1_o_mem_1.address, filtered_im_1_o_buff.physical_address)
mem_write.write(mem_write.register_map.filtered_real_1_o_mem_1.address, filtered_real_1_o_buff.physical_address)



# Start read and write
start = time.time()
mem_read.register_map.CTRL.AP_START = 1
mem_write.register_map.CTRL.AP_START = 1
end = time.time()
    
# Active wait
while mem_write.register_map.CTRL.AP_DONE == 0:
    end = time.time()
print ("time: ", end - start)
    
# TODO check results
#plot Histo
raw_real_vect = np.array(raw_data_real_o_buff)
plt.hist(raw_real_vect,bins='auto')
plt.title('Real component')
pltH = plt.gcf()
pltH.set_size_inches([10, 8])

raw_real_vect = np.array(raw_data_im_o_buff)
plt.hist(raw_real_vect,bins='auto')
plt.title('Imaginary component')
pltH = plt.gcf()
pltH.set_size_inches([10, 8])

#plot threshold
mad_real_vect = np.array(mad_R_o_buff)
std_real_vect = np.array(std_R_o_buff)
raw_real_vect2 = np.array(raw_data_real_1_o_buff)
plt.title('Real component')
plt.plot(raw_real_vect2,label='Signal')
plt.plot(mad_real_vect,label='MAD 3-$\sigma$ upper threshold')
plt.plot(-mad_real_vect,label='MAD 3-$\sigma$ lower threshold')
plt.plot(std_real_vect,label='STD 3-$\sigma$ upper threshold')
plt.plot(-std_real_vect,label='STD 3-$\sigma$ lower threshold')
pltH = plt.gcf()
pltH.set_size_inches([10, 8])
plt.legend()
plt.show()

mad_im_vect = np.array(mad_I_o_buff)
std_im_vect = np.array(std_I_o_buff)
raw_im_vect2 = np.array(raw_data_im_1_o_buff)
plt.title('Imaginary component')
plt.plot(raw_im_vect2,label='Signal')
plt.plot(mad_im_vect,label='MAD 3-$\sigma$ upper threshold')
plt.plot(-mad_im_vect,label='MAD 3-$\sigma$ lower threshold')
plt.plot(std_im_vect,label='STD 3-$\sigma$ upper threshold')
plt.plot(-std_im_vect,label='STD 3-$\sigma$ lower threshold')
pltH = plt.gcf()
pltH.set_size_inches([10, 8])
plt.legend()
plt.show()

#plot filtered signal
filter_real_vect = np.array(filtered_real_0_o_buff)
plt.title('Real component')
plt.plot(filter_real_vect,label='Filtered signal')
pltH = plt.gcf()
pltH.set_size_inches([10, 8])

filter_im_vect = np.array(filtered_im_0_o_buff)
plt.title('Imaginary component')
plt.plot(filter_im_vect,label='Filtered signal')
pltH = plt.gcf()
pltH.set_size_inches([10, 8])

# Data restitution
#TODO

ModuleNotFoundError: No module named 'numpy'